In [1]:
import pandas as pd
import numpy as np
import math
import h5py
import tempfile
import seaborn as sns
from qkeras import *
from qkeras import utils
import tensorflow as tf
from sklearn.model_selection import train_test_split
print(tf.__version__)
from tensorflow.keras.models import Sequential, Model,model_from_json
from keras.layers import Input,Dense
from keras import initializers, regularizers, optimizers, losses
from nn_globals import *
from nn_encode_displ import nlayers, nvariables
from nn_models import (create_model, create_model_sequential_bn2, create_model_pruned, 
                       create_model_sequential, create_model_sequential_bn,
                       lr_decay, modelbestcheck, modelbestcheck_weights)
from nn_training import train_model
from nn_models import load_my_model, update_keras_custom_objects
from keras.models import Model
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Setup matplotlib
plt.style.use('tdrstyle.mplstyle')

from nn_plotting import (gaus, fit_gaus, np_printoptions, \
                         find_efficiency_errors)

eps = 1e-7
my_cmap = plt.cm.viridis
my_cmap.set_under('w',1)
my_palette = ("#377eb8", "#e41a1c", "#984ea3", "#ff7f00", "#4daf4a")

[INFO    ] Using numpy 1.21.0
[INFO    ] Using tensorflow 2.4.1
[INFO    ] Using keras 2.4.3
[INFO    ] Using scipy 1.7.0
[INFO    ] Using sklearn 0.24.2


2.4.1


<ipython-input-1-86818f11bff2>:33: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("viridis"))
  my_cmap.set_under('w',1)


In [2]:
class evaluate:
    def __init__(self,X_test,y_test):
        self.X = X_test
        self.y = y_test[0]
        self.dxy = y_test[1]
    
    def compute_data_statistics(self,ctype = "y",label="data"):
        if ctype == "y":
            x = self.recalibrate(self.y,reg_pt_scale)
            x = x**(-1)
        else:
            x = self.recalibrate(self.dxy,reg_dxy_scale)
        df_describe = pd.DataFrame(x, columns = [label])
        print(df_describe.describe())
    
    def rmse(self,y_true, y_predicted):
        assert(y_true.shape[0] == y_predicted.shape[0])
        n = y_true.shape[0]
        sum_square = np.sum((y_true - y_predicted)**2)
        return math.sqrt(sum_square/n)
    
    def adjusted_r_2(self,y_true, y_predicted):
        y_addC = sm.add_constant(y_true)
        result = sm.OLS(y_predicted, y_addC).fit()
        print(result.rsquared, result.rsquared_adj)

    def recalibrate(self,x,scale):
        return x/scale
    
    def inverse(self,arr):
        arr_inv = 1./arr
        arr_inv[arr_inv == np.inf] = 0.
        return arr_inv
    
    def predict(self,model,batch_size = 256):
        y_test_true = self.recalibrate(self.y,reg_pt_scale)
        dxy_test_true = self.recalibrate(self.dxy, reg_dxy_scale)
        
        y_test = model.predict(self.X,batch_size = 2000)
        y_test_meas = y_test[:,0]
        dxy_test_meas = y_test[:,1]
        y_test_meas = self.recalibrate(y_test_meas,reg_pt_scale)
        dxy_test_meas = self.recalibrate(dxy_test_meas,reg_dxy_scale)   
    
        y_test_meas = y_test_meas.reshape(-1)
        dxy_test_meas = dxy_test_meas.reshape(-1)

        return y_test_meas, dxy_test_meas
    
    def compute_error(self,y_predicted,ctype = "y"):
        if ctype == "y":
            y_test_true = self.recalibrate(self.y,reg_pt_scale)
            print("RMSE Error for momentum:",self.rmse(self.inverse(y_test_true),\
                                                                              self.inverse(y_predicted)))
        else:
            dxy_test_true = self.recalibrate(self.dxy, reg_dxy_scale)
            print("RMSE Error for dxy:",self.rmse(dxy_test_true,y_predicted))

    def get_error(self,y_predicted,ctype = "y"):
        if ctype == "y":
            y_test_true = self.recalibrate(self.y,reg_pt_scale)
            return self.rmse(self.inverse(y_test_true),self.inverse(y_predicted))
        else:
            dxy_test_true = self.recalibrate(self.dxy, reg_dxy_scale)
            return self.rmse(dxy_test_true,y_predicted)

def k_fold_validation(model, x, y, dxy, folds =10):
    x_copy = np.copy(x)
    y_copy = np.copy(y)
    dxy_copy = np.copy(dxy)
    assert x_copy.shape[0] == y_copy.shape[0] == dxy_copy.shape[0]
    fold_size = int(x_copy.shape[0] / folds)
    x_splits, y_splits, dxy_splits = [], [], []
    for i in range(folds):
        indices = np.random.choice(x_copy.shape[0],fold_size, replace=False)  
        x_splits.append(x_copy[indices])
        y_splits.append(y_copy[indices])
        dxy_splits.append(dxy_copy[indices])
        x_copy = np.delete(x_copy,indices,axis = 0)
        y_copy = np.delete(y_copy,indices,axis = 0)
        dxy_copy = np.delete(dxy_copy,indices,axis = 0)
    rmse_y, rmse_dxy = [],[]
    for i in range(folds):
        evaluate_obj = evaluate(x_splits[i], tuple([y_splits[i],dxy_splits[i]]))
        y_predicted , dxy_predicted = evaluate_obj.predict(model = model)
        rmse_y.append(evaluate_obj.get_error(y_predicted,ctype="y"))
        rmse_dxy.append(evaluate_obj.get_error(dxy_predicted,ctype="dxy"))
    print('Average RMSE for '+ str(folds) + '-fold cv for y:', np.mean(rmse_y))
    print('Average RMSE for '+ str(folds) + '-fold cv for dxy:', np.mean(rmse_dxy))
    
def huber_loss(y_true, y_pred, delta=1.345):
    x = K.abs(y_true - y_pred)
    squared_loss = 0.5*K.square(x)
    absolute_loss = delta * (x - 0.5*delta)
    #xx = K.switch(x < delta, squared_loss, absolute_loss)
    xx = tf.where(x < delta, squared_loss, absolute_loss)  # needed for tensorflow
    return K.mean(xx, axis=-1)

def get_sparsity(weights):
    """
    Code borrowed from https://github.com/google/qkeras/blob/master/qkeras/utils.py#L937
    Returns the sparsity as the ratio of non-zero weights to the total weights within the weights matrix.
    """
    return 1.0 - np.count_nonzero(weights) / float(weights.size)

In [3]:
infile_muon_displ = "NN_input_params_FlatXYZ.npz"

nentries = 100000000

def _handle_nan_in_x(x):
    x[np.isnan(x)] = 0.0
    x[x==-999.0] = 0.0
    return x

def _zero_out_x(x):
    x = 0.0
    return x
    
def _fixME1Ring(x):
    for i in range(len(x)):
        if (x[i,0] != 0.0): x[i,18] = x[i,18] + 1
    return x   

def muon_data(filename, reg_pt_scale=1.0, reg_dxy_scale=1.0, correct_for_eta=False):
    try:
        logger.info('Loading muon data from {0} ...'.format(filename))
        loaded = np.load(filename)
        the_variables = loaded['variables']
        the_parameters = loaded['parameters']
        # print(the_variables.shape)
        the_variables = the_variables[:nentries]
        the_parameters = the_parameters[:nentries]
        logger.info('Loaded the variables with shape {0}'.format(the_variables.shape))
        logger.info('Loaded the parameters with shape {0}'.format(the_parameters.shape))
    except:
        logger.error('Failed to load data from file: {0}'.format(filename))

    assert(the_variables.shape[0] == the_parameters.shape[0])
    _handle_nan_in_x(the_variables)
      #_fixME1Ring(the_variables)
    _handle_nan_in_x(the_parameters)
    mask = np.logical_or(np.logical_or( np.logical_or((the_variables[:,23] == 11), (the_variables[:,23] == 13)), (the_variables[:,23] == 14)),(the_variables[:,23] == 15)) 

    the_variables = the_variables[mask]  
    the_parameters = the_parameters[mask]  
    assert(the_variables.shape[0] == the_parameters.shape[0])

    x = the_variables[:,0:23]
    y = reg_pt_scale*the_parameters[:,0]
#    print (x[0:30,:], the_variables[0:30,23])
#    print (y[0:30])
    phi = the_parameters[:,1] 
    eta = the_parameters[:,2] 
    vx = the_parameters[:,3] 
    vy = the_parameters[:,4] 
    vz = the_parameters[:,5]      
    dxy = vy * np.cos(phi) - vx * np.sin(phi) 
    dz = vz
    w = np.abs(y)/0.2 + 1.0
    x_mask = the_parameters[:,5]
    x_road = the_parameters[:,5] 
    _zero_out_x(x_mask)
    _zero_out_x(x_road)  
    logger.info('Loaded the encoded variables with shape {0}'.format(x.shape))
    logger.info('Loaded the encoded parameters with shape {0}'.format(y.shape))

    return x, y, dxy, dz, w, x_mask, x_road

def muon_data_split(filename, reg_pt_scale=1.0, reg_dxy_scale=1.0, test_size=0.5, correct_for_eta=False):
    x, y, dxy, dz, w, x_mask, x_road = muon_data(filename, reg_pt_scale=reg_pt_scale, reg_dxy_scale=reg_dxy_scale, correct_for_eta=correct_for_eta)

    # Split dataset in training and testing
    x_train, x_test, y_train, y_test, dxy_train, dxy_test, dz_train, dz_test, w_train, w_test, x_mask_train, x_mask_test, x_road_train, x_road_test = train_test_split(x, y, dxy, dz, w, x_mask, x_road, test_size=test_size)
    logger.info('Loaded # of training and testing events: {0}'.format((x_train.shape[0], x_test.shape[0])))

    # Check for cases where the number of events in the last batch could be too few
    validation_split = 0.1
    train_num_samples = int(x_train.shape[0] * (1.0-validation_split))
    val_num_samples = x_train.shape[0] - train_num_samples
    batch_size = 128
    if (train_num_samples%batch_size) < 100:
        logger.warning('The last batch for training could be too few! ({0}%{1})={2}. Please change test_size.'.format(train_num_samples, batch_size, train_num_samples%batch_size))
        logger.warning('Try this formula: int(int({0}*{1})*{2}) % 128'.format(x.shape[0], 1.0-test_size, 1.0-validation_split))
    train_num_samples = int(x_train.shape[0] * 2 * (1.0-validation_split))
    val_num_samples = x_train.shape[0] - train_num_samples
    batch_size = 128
    if (train_num_samples%batch_size) < 100:
        logger.warning('The last batch for training after mixing could be too few! ({0}%{1})={2}. Please change test_size.'.format(train_num_samples, batch_size, train_num_samples%batch_size))
        logger.warning('Try this formula: int(int({0}*{1})*2*{2}) % 128'.format(x.shape[0], 1.0-test_size, 1.0-validation_split))
    return x_train, x_test, y_train, y_test, dxy_train, dxy_test, dz_train, dz_test, w_train, w_test, x_mask_train, x_mask_test, x_road_train, x_road_test

In [4]:
# Import muon data
# 'x' is the array of input variables, 'y' is the q/pT
x_train_displ, x_test_displ, y_train_displ, y_test_displ, dxy_train_displ, dxy_test_displ, dz_train_displ, dz_test_displ, \
w_train_displ, w_test_displ, x_mask_train_displ, x_mask_test_displ, x_road_train_displ, x_road_test_displ = \
      muon_data_split(infile_muon_displ, reg_pt_scale=reg_pt_scale, reg_dxy_scale=reg_dxy_scale, test_size=0.315)

y_train_displ = np.abs(y_train_displ)
y_test_displ = np.abs(y_test_displ)


[INFO    ] Loading muon data from NN_input_params_FlatXYZ.npz ...
[INFO    ] Loaded the variables with shape (19300000, 25)
[INFO    ] Loaded the parameters with shape (19300000, 6)
[INFO    ] Loaded the encoded variables with shape (3284620, 23)
[INFO    ] Loaded the encoded parameters with shape (3284620,)
[INFO    ] Loaded # of training and testing events: (2249964, 1034656)
[WARNING ] The last batch for training could be too few! (2024967%128)=7. Please change test_size.
[WARNING ] Try this formula: int(int(3284620*0.685)*0.9) % 128
[WARNING ] The last batch for training after mixing could be too few! (4049935%128)=15. Please change test_size.
[WARNING ] Try this formula: int(int(3284620*0.685)*2*0.9) % 128


### Pre-Quantization Data Analysis:

In [5]:
# # Analysing dependencies in the data
# x = np.concatenate((x_train_displ,x_test_displ),axis=0)
# y = np.concatenate((y_train_displ,y_test_displ),axis=0)
# dxy = np.concatenate((dxy_train_displ,dxy_test_displ),axis=0)

# df = pd.DataFrame(x, columns = ["dphi_1","dphi_2","dphi_3","dphi_4","dphi_5","dphi_6",
#                                            "dtheta_1","dtheta_2","dtheta_3","dtheta_4","dtheta_5", "dtheta_6",
#                                            "bend_1","bend_2","bend_3","bend_4",
#                                            "FR","track theta","ME11",
#                                            "RPC_1","RPC_2","RPC_3","RPC_4"])
# df["Target momentum"] = y
# df["Target dxy"] = dxy
# # C_mat = df[["dphi_1","dphi_2","dphi_3","dphi_4","dphi_5","dphi_6",
# #             "dtheta_1","dtheta_2","dtheta_3","dtheta_4","dtheta_5", "dtheta_6",
# #            "track theta","Target momentum", "Target dxy"]].corr()
# C_mat = df.corr()
# fig = plt.figure(figsize = (7,7),dpi = 100)
# sns.heatmap(C_mat, vmax = .8, square = True,cmap = "YlGnBu")
# plt.show()

In [6]:
# y = np.concatenate((y_train_displ, y_test_displ))
# dxy = np.concatenate((dxy_train_displ, dxy_test_displ))
# assert(y.shape[0] == dxy.shape[0])
# stats = evaluate(x_train_displ,tuple([y,dxy]))
# stats.compute_data_statistics(ctype="y",label = "momentum")

In [7]:
# stats.compute_data_statistics(ctype="dxy",label="dxy")

### Defining the Quantized Model:

In [8]:
def create_model_quantized(nvariables, lr=0.001, clipnorm=10., initializer = "he_uniform", 
                           nodes1=64, nodes2=32, nodes3=16, outnodes=2):
    regularizer = regularizers.L1L2(l1=l1_reg, l2=l2_reg)
    bn_momentum = 0.9
    eps = 1e-4
    x = x_in = Input((nvariables,))
    x = QBatchNormalization(epsilon=eps, momentum=bn_momentum,name="bn-input")(x)
    
    x = QDense(nodes1, 
               kernel_quantizer="quantized_bits(bits=16,integer=6,symmetric=0,use_stochastic_rounding=True,alpha = \"auto\")",
               kernel_initializer=initializer,
               use_bias = True,
               kernel_regularizer = regularizer,
               name="hidden-dense-1")(x)
    x = QBatchNormalization(epsilon=eps, 
                            momentum=bn_momentum,
                            beta_quantizer='quantized_po2(bits = 6,max_value = 64, use_stochastic_rounding=True)',
                            gamma_quantizer='quantized_relu_po2(bits = 6,max_value = 64, use_stochastic_rounding=True)',
                            mean_quantizer='quantized_relu_po2(bits = 6,max_value = 64, use_stochastic_rounding=True)',
                            name="bn-1")(x)
    x = QActivation(activation="quantized_relu(bits=16,integer=6,use_sigmoid=True,use_stochastic_rounding=True)",
                    name="act_1")(x)
    
    if nodes2:
    
        x = QDense(nodes2, 
                   kernel_quantizer="quantized_bits(bits=16,integer=6,symmetric=0,use_stochastic_rounding=True,alpha = \"auto\")", 
                   kernel_initializer=initializer,
                   use_bias = True,
                   kernel_regularizer = regularizer,
                   name="hidden-dense-2")(x)
        x = QBatchNormalization(epsilon=eps, 
                                momentum=bn_momentum,
                                beta_quantizer='quantized_po2(bits = 6,max_value = 64, use_stochastic_rounding=True)',
                                gamma_quantizer='quantized_relu_po2(bits = 6,max_value = 64, use_stochastic_rounding=True)',
                                name="bn-2")(x)
        x = QActivation(activation="quantized_relu(bits=18,integer=9, use_sigmoid=True,use_stochastic_rounding=True)", 
                        name="act_2")(x)
        
        if nodes3:

            x = QDense(nodes3, 
                       kernel_quantizer="quantized_bits(bits=16,integer=6,symmetric=0,use_stochastic_rounding=True,alpha = \"auto\")", 
                       kernel_initializer=initializer,
                       kernel_regularizer = regularizer,
                       use_bias = True,
                       name="hidden-dense-3")(x)
            x = QBatchNormalization(epsilon=eps, 
                                    momentum=bn_momentum,
                                    beta_quantizer='quantized_po2(bits = 6,max_value = 64, use_stochastic_rounding=True)',
                                    gamma_quantizer='quantized_relu_po2(bits = 6,max_value = 64, use_stochastic_rounding=True)',
                                    name="bn-3")(x)
            x = QActivation(activation="quantized_relu(bits=16,integer=6, use_sigmoid=True,use_stochastic_rounding=True)", 
                            name="act_3")(x)

    x = QDense(outnodes,
                kernel_quantizer="quantized_bits(bits=16,integer=6,symmetric=0,keep_negative = False,use_stochastic_rounding=True)",
                kernel_initializer = initializer,
                use_bias = True,
                name="dense-output")(x)
    x = Activation("linear")(x)
    
    model = Model(inputs=x_in, outputs=x,name="qmodel")
    
    adam = optimizers.Adam(lr=lr, clipnorm=clipnorm)
    model.compile(optimizer=adam, loss=huber_loss, metrics=['accuracy'])
    model.summary()
    
    return model
    

In [ ]:
assert(keras.backend.backend() == 'tensorflow')

# Bits = 24, Int = 12
# normal_epochs = 300
# normal_batch_size = 1000
# l1_reg = 1e-5
# l2_reg = 0.0
# learning_rate = 1e-3

# Bits = 18, Int = 12
# normal_epochs = 300
# normal_batch_size = 500
# l1_reg = 1e-2
# l2_reg = 0.0
# learning_rate = 5e-3

# Bits = 18, Int = 6
# normal_epochs = 300
# normal_batch_size = 500
# l1_reg = 1e-5
# l2_reg = 0.0
# learning_rate = 5e-3

# Bits = 12, Int = 6
# normal_epochs = 300
# normal_batch_size = 1000
# l1_reg = 1e-3
# l2_reg = 0.0
# learning_rate = 1e-3

# Bits = 10, Int = 5
normal_epochs = 300
normal_batch_size = 1000
l1_reg = 0.0
l2_reg = 0.0
learning_rate = 1e-3

qmodel = create_model_quantized(nvariables=nvariables, lr=learning_rate, clipnorm=gradient_clip_norm, 
                              nodes1=20, nodes2=15, nodes3=10, outnodes=2)
logger.info('Training model with l1_reg: {0} l2_reg: {1}'.format(l1_reg, l2_reg))

history = train_model(qmodel, x_train_displ, np.column_stack((y_train_displ, dxy_train_displ)),
                    save_model=False, epochs=normal_epochs, batch_size=normal_batch_size,
#                     callbacks=[lr_decay,modelbestcheck,modelbestcheck_weights], 
                    validation_split=0.1, verbose=True)

metrics = [len(history.history['loss']), history.history['loss'][-1], history.history['val_loss'][-1]]
logger.info('Epoch {0}/{0} - loss: {1} - val_loss: {2}'.format(*metrics))

[INFO    ] Training model with l1_reg: 0.0 l2_reg: 0.0
[INFO    ] Begin training ...


Model: "qmodel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
bn-input (QBatchNormalizatio (None, 23)                92        
_________________________________________________________________
hidden-dense-1 (QDense)      (None, 20)                480       
_________________________________________________________________
bn-1 (QBatchNormalization)   (None, 20)                80        
_________________________________________________________________
act_1 (QActivation)          (None, 20)                0         
_________________________________________________________________
hidden-dense-2 (QDense)      (None, 15)                315       
_________________________________________________________________
bn-2 (QBatchNormalization)   (None, 15)                60   

2025/2025 [==============================] - 5s 2ms/step - loss: 11.8106 - accuracy: 0.9322 - val_loss: 12.8093 - val_accuracy: 0.9319

Epoch 00018: val_loss did not improve from 11.03439

Epoch 00018: val_loss did not improve from 11.03439
Epoch 19/300

Epoch 00019: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
2025/2025 [==============================] - 5s 2ms/step - loss: 12.0369 - accuracy: 0.9292 - val_loss: 11.3131 - val_accuracy: 0.9270

Epoch 00019: val_loss did not improve from 11.03439

Epoch 00019: val_loss did not improve from 11.03439
Epoch 20/300

Epoch 00020: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
2025/2025 [==============================] - 5s 2ms/step - loss: 11.6203 - accuracy: 0.9309 - val_loss: 11.0632 - val_accuracy: 0.9349

Epoch 00020: val_loss did not improve from 11.03439

Epoch 00020: val_loss did not improve from 11.03439
Epoch 21/300

Epoch 00021: LearningRateScheduler reducing learning rate to 0.0008

2025/2025 [==============================] - 5s 2ms/step - loss: 11.4575 - accuracy: 0.9327 - val_loss: 10.9436 - val_accuracy: 0.9338

Epoch 00042: val_loss did not improve from 10.87763

Epoch 00042: val_loss did not improve from 10.87763
Epoch 43/300

Epoch 00043: LearningRateScheduler reducing learning rate to 0.0006561000482179224.
2025/2025 [==============================] - 5s 2ms/step - loss: 11.4424 - accuracy: 0.9329 - val_loss: 10.9910 - val_accuracy: 0.9352

Epoch 00043: val_loss did not improve from 10.87763

Epoch 00043: val_loss did not improve from 10.87763
Epoch 44/300

Epoch 00044: LearningRateScheduler reducing learning rate to 0.0006561000482179224.
2025/2025 [==============================] - 5s 3ms/step - loss: 11.3860 - accuracy: 0.9334 - val_loss: 10.9900 - val_accuracy: 0.9330

Epoch 00044: val_loss did not improve from 10.87763

Epoch 00044: val_loss did not improve from 10.87763
Epoch 45/300

Epoch 00045: LearningRateScheduler reducing learning rate to 0.0006

2025/2025 [==============================] - 5s 2ms/step - loss: 12.1174 - accuracy: 0.9311 - val_loss: 11.5880 - val_accuracy: 0.9342

Epoch 00066: val_loss did not improve from 10.87763

Epoch 00066: val_loss did not improve from 10.87763
Epoch 67/300

Epoch 00067: LearningRateScheduler reducing learning rate to 0.0005314410664141178.
2025/2025 [==============================] - 5s 2ms/step - loss: 12.1257 - accuracy: 0.9312 - val_loss: 16.2555 - val_accuracy: 0.9102

Epoch 00067: val_loss did not improve from 10.87763

Epoch 00067: val_loss did not improve from 10.87763
Epoch 68/300

Epoch 00068: LearningRateScheduler reducing learning rate to 0.0005314410664141178.
2025/2025 [==============================] - 5s 2ms/step - loss: 12.5681 - accuracy: 0.9294 - val_loss: 14.1417 - val_accuracy: 0.9252

Epoch 00068: val_loss did not improve from 10.87763

Epoch 00068: val_loss did not improve from 10.87763
Epoch 69/300

Epoch 00069: LearningRateScheduler reducing learning rate to 0.0005

...

2025/2025 [==============================] - 5s 3ms/step - loss: 11.9860 - accuracy: 0.9289 - val_loss: 11.6384 - val_accuracy: 0.9299

Epoch 00186: val_loss did not improve from 10.77507

Epoch 00186: val_loss did not improve from 10.77507
Epoch 187/300

Epoch 00187: LearningRateScheduler reducing learning rate to 0.000150094652781263.
2025/2025 [==============================] - 5s 3ms/step - loss: 11.9599 - accuracy: 0.9286 - val_loss: 11.7764 - val_accuracy: 0.9295

Epoch 00187: val_loss did not improve from 10.77507

Epoch 00187: val_loss did not improve from 10.77507
Epoch 188/300

Epoch 00188: LearningRateScheduler reducing learning rate to 0.000150094652781263.
2025/2025 [==============================] - 5s 3ms/step - loss: 11.9094 - accuracy: 0.9290 - val_loss: 11.8107 - val_accuracy: 0.9271

Epoch 00188: val_loss did not improve from 10.77507

Epoch 00188: val_loss did not improve from 10.77507
Epoch 189/300

Epoch 00189: LearningRateScheduler reducing learning rate to 0.000

2025/2025 [==============================] - 5s 3ms/step - loss: 11.5864 - accuracy: 0.9305 - val_loss: 11.1222 - val_accuracy: 0.9331

Epoch 00210: val_loss did not improve from 10.77507

Epoch 00210: val_loss did not improve from 10.77507
Epoch 211/300

Epoch 00211: LearningRateScheduler reducing learning rate to 0.00010941899454337544.
2025/2025 [==============================] - 5s 3ms/step - loss: 11.6282 - accuracy: 0.9309 - val_loss: 13.4399 - val_accuracy: 0.9204

Epoch 00211: val_loss did not improve from 10.77507

Epoch 00211: val_loss did not improve from 10.77507
Epoch 212/300

Epoch 00212: LearningRateScheduler reducing learning rate to 0.00010941899381577969.
2025/2025 [==============================] - 5s 3ms/step - loss: 11.7434 - accuracy: 0.9299 - val_loss: 11.1436 - val_accuracy: 0.9343

Epoch 00212: val_loss did not improve from 10.77507

Epoch 00212: val_loss did not improve from 10.77507
Epoch 213/300

Epoch 00213: LearningRateScheduler reducing learning rate to 0

2025/2025 [==============================] - 5s 3ms/step - loss: 11.5013 - accuracy: 0.9320 - val_loss: 11.2710 - val_accuracy: 0.9311

Epoch 00234: val_loss did not improve from 10.77507

Epoch 00234: val_loss did not improve from 10.77507
Epoch 235/300

Epoch 00235: LearningRateScheduler reducing learning rate to 8.862938557285815e-05.
2025/2025 [==============================] - 5s 3ms/step - loss: 11.4944 - accuracy: 0.9317 - val_loss: 10.7779 - val_accuracy: 0.9356

Epoch 00235: val_loss did not improve from 10.77507

Epoch 00235: val_loss did not improve from 10.77507
Epoch 236/300

Epoch 00236: LearningRateScheduler reducing learning rate to 8.862938557285815e-05.
2025/2025 [==============================] - 5s 3ms/step - loss: 11.5091 - accuracy: 0.9313 - val_loss: 11.5777 - val_accuracy: 0.9213

Epoch 00236: val_loss did not improve from 10.77507

Epoch 00236: val_loss did not improve from 10.77507
Epoch 237/300

Epoch 00237: LearningRateScheduler reducing learning rate to 8.8

2025/2025 [==============================] - 5s 3ms/step - loss: 11.6718 - accuracy: 0.9308 - val_loss: 11.2280 - val_accuracy: 0.9336

Epoch 00258: val_loss did not improve from 10.77507

Epoch 00258: val_loss did not improve from 10.77507
Epoch 259/300

Epoch 00259: LearningRateScheduler reducing learning rate to 7.178980013122782e-05.
2025/2025 [==============================] - 6s 3ms/step - loss: 11.6912 - accuracy: 0.9309 - val_loss: 11.1433 - val_accuracy: 0.9350

Epoch 00259: val_loss did not improve from 10.77507

Epoch 00259: val_loss did not improve from 10.77507
Epoch 260/300

Epoch 00260: LearningRateScheduler reducing learning rate to 7.178980013122782e-05.
2025/2025 [==============================] - 6s 3ms/step - loss: 11.6445 - accuracy: 0.9312 - val_loss: 11.0005 - val_accuracy: 0.9338

Epoch 00260: val_loss did not improve from 10.77507

Epoch 00260: val_loss did not improve from 10.77507
Epoch 261/300

Epoch 00261: LearningRateScheduler reducing learning rate to 6.4

2025/2025 [==============================] - 5s 3ms/step - loss: 11.6627 - accuracy: 0.9316 - val_loss: 11.2324 - val_accuracy: 0.9333

Epoch 00282: val_loss did not improve from 10.77507

Epoch 00282: val_loss did not improve from 10.77507
Epoch 283/300

Epoch 00283: LearningRateScheduler reducing learning rate to 5.233476622379385e-05.
2025/2025 [==============================] - 5s 3ms/step - loss: 11.6431 - accuracy: 0.9315 - val_loss: 12.0531 - val_accuracy: 0.9300

Epoch 00283: val_loss did not improve from 10.77507

Epoch 00283: val_loss did not improve from 10.77507
Epoch 284/300

Epoch 00284: LearningRateScheduler reducing learning rate to 5.233476622379385e-05.
2025/2025 [==============================] - 6s 3ms/step - loss: 11.6471 - accuracy: 0.9317 - val_loss: 11.8441 - val_accuracy: 0.9267

Epoch 00284: val_loss did not improve from 10.77507

Epoch 00284: val_loss did not improve from 10.77507
Epoch 285/300

Epoch 00285: LearningRateScheduler reducing learning rate to 5.2

In [ ]:
x = np.concatenate((x_train_displ, x_test_displ),axis = 0)
y = np.concatenate((y_train_displ, y_test_displ),axis = 0)
dxy = np.concatenate((dxy_train_displ, dxy_test_displ),axis = 0)
k_fold_validation(model = qmodel, 
                  x = x,  
                  y = y, 
                  dxy = dxy, 
                  folds =50)

In [ ]:
plt.figure(figsize=(5,5),dpi = 70)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('quantized model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
training_params = {"retrain epochs":normal_epochs, 
                   "batch size"    :normal_batch_size, 
                   "learning rate" :learning_rate,
                   "BN momentum"   :0.9,
                   "BN epsilon"    :1e-4,
                   "L1 Reg"        :l1_reg,
                   "L2 Reg"        :l2_reg
                  }
df = pd.DataFrame(training_params.items(), 
                  columns=['Training Parameters', 'Value'],
                  index = [i+1 for i in range(len(training_params))])
df

In [ ]:
def __generate_delta_plots__(model,
                             x,
                             y,
                             dxy,batch_size = 4096):
    
    # Predictions
    y_test_true = y.copy()
    y_test_true /= reg_pt_scale

    y_test_sel = (np.abs(1.0/y) >= 20./reg_pt_scale)

    y_test_meas_ = model.predict(x, batch_size=4096)
    y_test_meas = y_test_meas_[:,0]
    y_test_meas /= reg_pt_scale
    y_test_meas = y_test_meas.reshape(-1)

    dxy_test_true = dxy.copy()
    dxy_test_true /= reg_dxy_scale

    dxy_test_sel = (np.abs(dxy_test_true) >= 25)

    dxy_test_meas = y_test_meas_[:,1]
    dxy_test_meas /= reg_dxy_scale
    dxy_test_meas = dxy_test_meas.reshape(-1)
    
    # Plot Delta(q/pT)/pT
    plt.figure(figsize=(5,5),dpi = 75)
    yy = ((np.abs(1.0/y_test_meas) - np.abs(1.0/y_test_true))/np.abs(1.0/y_test_true))
    hist, edges, _ = plt.hist(yy, bins=100, range=(-2.0,2.0-eps), histtype='stepfilled', facecolor='c', alpha=0.6)
    plt.xlabel(r'$\Delta(p_{T})_{\mathrm{meas-true}}/{(p_{T})}_{true}$ [1/GeV]')
    plt.ylabel(r'entries')
    logger.info('# of entries: {0}, mean: {1}, std: {2}'.format(len(yy), np.mean(yy), np.std(yy[np.abs(yy)<0.4])))

    popt = fit_gaus(hist, edges, mu=np.mean(yy), sig=np.std(yy[np.abs(yy)<2.0]))
    logger.info('gaus fit (a, mu, sig): {0}'.format(popt))
    xdata = (edges[1:] + edges[:-1])/2
    plt.plot(xdata, gaus(xdata, popt[0], popt[1], popt[2]), color='c')
    plt.show()

    # Plot Delta(dxy)
    plt.figure(figsize=(5,5),dpi = 75)
    yy = (dxy_test_meas - dxy_test_true)[y_test_sel]
    hist, edges, _ = plt.hist(yy, bins=100, range=(-50,50), histtype='stepfilled', facecolor='c', alpha=0.6)
    plt.xlabel(r'$\Delta(d_{0})_{\mathrm{meas-true}}$ [cm]')
    plt.ylabel(r'entries')
    logger.info('# of entries: {0}, mean: {1}, std: {2}'.format(len(yy), np.mean(yy), np.std(yy)))

    popt = fit_gaus(hist, edges, mu=np.mean(yy), sig=np.std(yy))
    logger.info('gaus fit (a, mu, sig): {0}'.format(popt))
    xdata = (edges[1:] + edges[:-1])/2
    plt.plot(xdata, gaus(xdata, popt[0], popt[1], popt[2]), color='c')
    plt.show()

In [ ]:
__generate_delta_plots__(model = qmodel,
                        x = x_test_displ,
                        y = y_test_displ,
                        dxy = dxy_test_displ)

### Saving the optimal quantized model:

In [ ]:
# model_for_export = qmodel
# qkeras_file = "model/quantized_DNN.h5"
# weights_file = "model/quantized_DNN_weights"
# # save quantized weights
# utils.model_save_quantized_weights(model_for_export, weights_file)
# # save keras.save() h5 version of quantized model
# tf.keras.models.save_model(model_for_export, qkeras_file, include_optimizer=False)
# print('Saved quantized Keras model to:', qkeras_file)

### Loading the quantized model for assessment:

In [ ]:
# qkeras_file = "model/quantized_DNN.h5"
# weights_file = "model/quantized_DNN_weights"
# loaded_qmodel = utils.load_qmodel(filepath=qkeras_file)

In [ ]:
# loaded_qmodel.load_weights(weights_file)

In [ ]:
# loaded_qmodel.summary()

In [ ]:
# utils.get_model_sparsity(model=loaded_qmodel,per_layer=True)

In [ ]:
## END ############################################################################################